In [1]:
#all imports
import carla #the sim library itself
import random #to pick random spawn point
import cv2 #to work with images from cameras
import numpy as np #in this example to change image representation - re-shaping

In [2]:
# connect to the sim 
client = carla.Client('localhost', 2000) #미리 실행해둔 서버에 접속

In [3]:
# optional to load different towns
#client.set_timeout(15)
client.load_world('Town04') # 맵 변경

RuntimeError: time-out of 5000ms while waiting for the simulator, make sure the simulator is ready and connected to localhost:2000

In [5]:
#define environment/world and get possible places to spawn a car
world = client.get_world() #현재 시뮬레이터를 world에 저장
spawn_points = world.get_map().get_spawn_points() #spawn: 생성, 객체를 만들어 시뮬레이터에 배치하는 과정, 그 위치를 spawn_points로 가져옴

In [11]:
#look for a blueprint of Mini car
vehicle_bp = world.get_blueprint_library().filter('*mini*') #blueprint library에서 mini 차량들을 가져옴

In [12]:
#spawn a car in a random location

start_point = spawn_points[0] # 여러 위치 리스트 중 0번째 위치 사용
vehicle = world.try_spawn_actor(vehicle_bp[0], start_point) #start_point에 actor를 spawn함, 차량 후보들 중 0번째 차량 사용.


In [13]:
# move simulator view to the car
spectator = world.get_spectator()
#관찰자 역할을 하는 spectator 객체를 가져옴. 자유롭게 이동하며 화면을 관찰하는 가상의 카메라 개념
#위치, 각도 조절 가능
start_point.location.z = start_point.location.z+1 #start_point was used to spawn the car but we move 1m up to avoid being on the floor
#z축 1 값만큼 위에서 생성됨.
spectator.set_transform(start_point) # 시작 위치로 관점 설정
#spectator위치를 특정한 곳으로 보낼 때 코드
#new_transform = carla.Transform(new_loc, new_rot)
#spectator.set_transform(new_transform)

In [13]:
#send the car off on autopilot - this will leave the spectator
vehicle.set_autopilot(True)

In [14]:
#카메라 세팅

#setting RGB Camera - this follow the approach explained in a Carla video
# link: https://www.youtube.com/watch?v=om8klsBj4rc&t=1184s

#camera mount offset on the car - you can tweak these to each car to avoid any parts of the car being in the view
#카메라 위치 설정, 각 차에 맞게 블랙박스 시점으로 맞출 수 있음
CAMERA_POS_Z = 1.6 #this means 1.6m up from the ground
CAMERA_POS_X = 0.9 #this is 0.9m forward

camera_bp = world.get_blueprint_library().find('sensor.camera.rgb') # blueprint library에서 RGB카메라 센서들을 가져옴
camera_bp.set_attribute('image_size_x', '640') # this ratio works in CARLA 9.14 on Windows # 가로해상도 640픽셀
camera_bp.set_attribute('image_size_y', '360') # 세로 해상도 360픽셀

camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X)) #위치 변환 객체
#this creates the camera in the sim
camera = world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)#카메라 센서를 생성해 특정 차량에 부착

def camera_callback(image,data_dict): # 카메라가 촬영한 raw data를 RGBA 형태의 이미지 array로 저장하는 함수
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

image_w = camera_bp.get_attribute('image_size_x').as_int() # camera blueprint에서 설정한 적정 이미지 크기를 불러옴
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image': np.zeros((image_h,image_w,4))}# 이미지를 저장할 빈 array생성
# this actually opens a live stream from the camera
camera.listen(lambda image: camera_callback(image,camera_data)) #listen 함수로 카메라에서 실시간으로 데이터를 받아올 수 있도록 스트림을 열고,
#매 프레임마다 camera_callback함수가 호출되어 영상을 camera_data에 저장

In [15]:
#계속 찍기
while True: 
      
    # Dispaly with imshow
    cv2.imshow('All cameras',camera_data['image']) 
    #실시간으로 camera_data dict에 저장된 이미지를 All cameras window에 보여줌
    
    # Break loop if user presses q
    if cv2.waitKey(1) == ord('q'):
        break
cv2.destroyAllWindows() #이미지 표시 창 모두 닫기

In [9]:
#grab a snapshot from the camera an show in a pop-up window
#스냅샷 찍기
img = camera_data['image'] #'image'를 img 변수에 저장
cv2.imshow('RGB Camera',img) # 저장된 이미지를 'RGB Camera'라는 팝업 창에 띄움
cv2.waitKey(0) #키보드 입력이 있을 때까지 창을 유지, 아무 키나 누르면 다음 단계로 넘어감

-1

In [10]:
# clean up after yourself

camera.stop() # this is the opposite of camera.listen 실시간 데이터 수신 중지
for actor in world.get_actors().filter('*vehicle*'): #모든 차량 객체를 찾으며 destroy
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'): #모든 센서 객체를 찾으며 destroy
    sensor.destroy()

In [7]:
len(spawn_points) #spawn포인트(차량이나 보행자가 생성될 수 있는 위치)의 총 개수

10422

In [8]:
start_point = spawn_points[0]
spectator = world.get_spectator()
spectator.set_transform(start_point) #다시 시작 위치나 관찰뷰 위치 설정하고

In [12]:
spectator.set_transform(carla.Transform(carla.Location(x=-1085.286377, y=3112.225830, z=356.060608), carla.Rotation(pitch=1.648070, yaw=20.234367, roll=0.000000)))

In [11]:
print(start_point)

Transform(Location(x=-1085.286377, y=3112.225830, z=356.060608), Rotation(pitch=1.648070, yaw=20.234367, roll=0.000000))
